In [101]:
import matplotlib.pyplot as plt
import numpy as np
import mysql.connector

import pandas as pd

mydb = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="FH_Master_2122",
    database="forschung"
)

mycursor = mydb.cursor()


In [104]:
mycursor = mydb.cursor()

mycursor.execute("SELECT * FROM diagnosen")
myresult = mycursor.fetchall()

df = pd.DataFrame(myresult, columns = ['id', 'klientid','giltab','giltbis','bezeichnung','code','status','statusname'])
diagnoses = df
a = df.bezeichnung.unique()
diabetes = []
for dia in list(a):
    try:
        if "diab" in dia:
            diabetes.append(dia)
    except:
        continue

#diabetes = ['Diabetes', 'Diab. mellitus', 'Diabetes mellitus', 'Diabetes mellitus Typ 2', 'Diabetes mellitus Typ II']
df_filter = df[df.bezeichnung.isin(diabetes)]
# 245 Klienten haben eine Krankheit mit Diabetes, es gibt 304 Diagnosen
klienten = list(df_filter.klientid.unique())

In [106]:
diagnoses

,id,klientid,giltab,giltbis,bezeichnung,code,status,statusname
0,1,718,2007-01-10,NaT,Hyertensive Krise bei art. Hypertonus,None,-1,Keine Angabe
1,2,718,2007-01-10,NaT,Rezidiv Vertigo mit Sturzneigung,None,-1,Keine Angabe
2,3,2681,2006-04-01,NaT,cerebeläre Ataxie,None,-1,Keine Angabe
3,4,2681,2006-04-01,NaT,Osteoporose,None,-1,Keine Angabe
4,5,420,1997-05-02,NaT,senile Demenz,None,-1,Keine Angabe
...,...,...,...,...,...,...,...,...
69867,69868,23488,2021-11-10,NaT,akute Niereninsuffizienz,None,-1,Keine Angabe
69868,69869,23488,2021-11-10,NaT,Subilieus bei Koprostase (10/2021),None,-1,Keine Angabe
69869,69870,23488,2021-11-10,NaT,COVID19-Infektion (nachgewiesen am 13.10.2021),None,-1,Keine Angabe
69870,69871,21910,2021-11-10,NaT,Pertrochantäre Femurfraktur links mit laterale...,None,-1,Keine Angabe


In [107]:
mycursor.execute("SELECT * FROM vitalwerte")
myresult = mycursor.fetchall()

vp = pd.DataFrame(myresult, columns = ['id', 'klientid','datum','bezeichnung','wert','einheit','wurdeunterschritten','wurdeueberschritten'])

In [108]:
vp_filter = vp[vp.klientid.isin(klienten)]
len(vp_filter.klientid.unique())
klienten = list(vp_filter.klientid.unique())

In [109]:
mycursor = mydb.cursor()

mycursor.execute("SELECT * FROM medikamentgaben")
myresult = mycursor.fetchall()

medi = pd.DataFrame(myresult, columns=['id','klientid','tag','datum','bezeichnung','pzn','menge','einheit','atccode'])

In [110]:
mycursor.execute("SELECT * FROM medikamentinhaltsstoffe")
myresult = mycursor.fetchall()
inhalt = pd.DataFrame(myresult, columns = ['id','pzn','bezeichnung','menge','einheit'])

In [111]:
# Blutzucker 175 Personen
# 140 Personen mit mehr als 12 Blutzuckerwerten
# 74 Personen mit mehr als 100 Blutzuckerwerten
vp_blutzucker = vp_filter[vp_filter.bezeichnung == 'Blutzucker']
anzahl = vp_blutzucker.klientid.value_counts()
klienten = list(anzahl[anzahl > 100].index)
len(klienten)

74

In [112]:
my_diagnoses = diagnoses[diagnoses["klientid"].isin(klienten)]

In [113]:
vp_filter_bz = vp_blutzucker[vp_blutzucker.klientid.isin(klienten)]

In [114]:
ergebnisse = {}
for klient in klienten:
    df = vp_filter_bz[vp_filter_bz.klientid == klient].sort_values('datum')
    datum = list(df.datum)
    beginn = datum[0]
    ende = datum[-1]
    anzahl = len(datum)
    ergebnisse[klient] = [beginn, ende, anzahl]
ergebnisse

{5934: [Timestamp('2009-05-04 07:30:00'),
  Timestamp('2020-10-20 06:30:00'),
  8889],
 9829: [Timestamp('2012-11-30 17:26:00'),
  Timestamp('2019-09-27 06:30:00'),
  8340],
 12079: [Timestamp('2014-08-06 22:00:00'),
  Timestamp('2021-11-10 22:00:00'),
  7924],
 10532: [Timestamp('2013-04-05 17:46:00'),
  Timestamp('2021-11-02 18:00:00'),
  7508],
 8711: [Timestamp('2011-08-23 17:29:00'),
  Timestamp('2019-01-26 18:00:00'),
  7408],
 4197: [Timestamp('2007-11-12 12:20:00'),
  Timestamp('2013-11-05 06:30:00'),
  6878],
 13277: [Timestamp('2016-07-17 11:12:00'),
  Timestamp('2021-06-17 18:00:00'),
  5250],
 7672: [Timestamp('2010-09-24 17:30:00'),
  Timestamp('2017-06-03 13:37:00'),
  5203],
 10773: [Timestamp('2013-06-06 18:00:00'),
  Timestamp('2018-08-01 06:30:00'),
  5058],
 12982: [Timestamp('2015-05-09 12:38:00'),
  Timestamp('2020-11-24 19:00:00'),
  4793],
 14242: [Timestamp('2016-04-05 06:30:00'),
  Timestamp('2021-11-11 06:30:00'),
  4154],
 19578: [Timestamp('2019-06-14 18:00:

In [115]:
ergebnisse = pd.DataFrame(ergebnisse)
ergebnisse = ergebnisse.transpose()
ergebnisse.columns = ["Anfang","Ende","Anzahl_Messungen"]
ergebnisse["Zeitspanne"] = (ergebnisse.Ende - ergebnisse.Anfang).round('D')
ergebnisse

# hohe Standardabweichung, Mittelwert = 1129 Tage, Median = 907, Quartilsbox = 332 - 1823

,Anfang,Ende,Anzahl_Messungen,Zeitspanne
5934,2009-05-04 07:30:00,2020-10-20 06:30:00,8889,4187 days
9829,2012-11-30 17:26:00,2019-09-27 06:30:00,8340,2492 days
12079,2014-08-06 22:00:00,2021-11-10 22:00:00,7924,2653 days
10532,2013-04-05 17:46:00,2021-11-02 18:00:00,7508,3133 days
8711,2011-08-23 17:29:00,2019-01-26 18:00:00,7408,2713 days
...,...,...,...,...
18220,2018-08-21 18:00:00,2018-11-08 06:30:00,127,79 days
20112,2019-09-16 21:38:00,2019-10-14 07:30:00,111,27 days
5509,2011-05-12 09:30:00,2018-01-03 08:54:00,105,2428 days
15682,2017-03-11 06:30:00,2017-04-13 21:14:00,102,34 days


In [116]:
my_vp = vp_filter[vp_filter.klientid.isin(klienten)]
my_medi = medi[medi.klientid.isin(klienten)]
my_dia = my_diagnoses


In [117]:
import numpy as np
werte = list(my_vp.wert.apply(str))
werte = [w.replace(',','.') for w in werte]
neu = [float(x.split("/")[0]) if len(x)>1 else np.NaN for x in werte]
my_vp["werte"] = neu

<ipython-input-117-9a984c4a286a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_vp["werte"] = neu


In [118]:
my_vp.to_csv("C:/Users/Caroline Kries/sciebo/Forschungsprojekt Medikamenteninteraktion/03_Daten/my_vp")
my_medi.to_csv("C:/Users/Caroline Kries/sciebo/Forschungsprojekt Medikamenteninteraktion/03_Daten/my_medi")
my_dia.to_csv("C:/Users/Caroline Kries/sciebo/Forschungsprojekt Medikamenteninteraktion/03_Daten/my_dia")

,id,klientid,datum,bezeichnung,wert,einheit,wurdeunterschritten,wurdeueberschritten,werte
953,954,4197,2011-04-26 07:45:00,Blutzucker,160,mg/dl,0,0,160.0
954,955,4197,2011-04-26 11:15:00,Blutzucker,175,mg/dl,0,0,175.0
955,956,4197,2011-04-26 17:15:00,Blutzucker,224,mg/dl,0,0,224.0
956,957,4197,2011-04-27 07:45:00,Blutzucker,182,mg/dl,0,0,182.0
957,958,4197,2011-04-27 11:15:00,Blutzucker,256,mg/dl,0,0,256.0
...,...,...,...,...,...,...,...,...,...
1072097,1072098,7957,2014-10-23 18:00:00,Blutzucker,124,mg/dl,0,0,124.0
1072098,1072099,7957,2014-10-23 23:44:00,Blutzucker,130,mg/dl,0,0,130.0
1072099,1072100,7957,2014-10-24 08:05:00,Blutzucker,110,mg/dl,0,0,110.0
1072100,1072101,7957,2014-10-24 11:30:00,Blutdruck,126/74,mm Hg,0,0,126.0
